In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import gensim
import mlflow
import mlflow.sklearn
from sklearn.metrics import classification_report

In [4]:
# Download stopwords if not already downloaded
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [5]:
data=pd.read_csv('D:\Model_Deployement\Dataset\Phishing_Email.csv',index_col=False)

In [6]:
data.head()

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [7]:
data.isna().sum()

Unnamed: 0     0
Email Text    16
Email Type     0
dtype: int64

In [8]:
# replace the na with empty space
data['Email Text'].fillna('',inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_18728\1110725904.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Email Text'].fillna('',inplace=True)


In [9]:
# function for clean the data 
ps=PorterStemmer()

def clean(data):
    data = re.sub(r'[^a-zA-Z\s]', '', data)
    data = data.lower()
    stop_words = set(stopwords.words('english'))
    words = data.split()
    words = [word for word in words if word not in stop_words]
    words = [ps.stem(word) for word in words]
    return words  # returns a list of tokens

In [10]:
record=data['Email Text'][0]

In [11]:
cleaned_record=clean(record)

In [12]:
data['tokens']=data['Email Text'].apply(clean)

In [13]:
data['tokens']

0        [disc, uniformitarian, sex, lang, dick, hudson...
1        [side, galicismo, galicismo, spanish, term, na...
2        [equistar, deal, ticket, still, avail, assist,...
3        [hello, hot, lil, horni, toy, one, dream, open...
4        [softwar, incred, low, price, lower, draperi, ...
                               ...                        
18645    [date, lone, housewif, alway, want, date, lone...
18646    [request, submit, access, request, anita, dupo...
18647    [import, prc, mtg, hi, dorn, john, discov, rec...
18648    [press, clip, letter, californian, util, pleas...
18649                                              [empti]
Name: tokens, Length: 18650, dtype: object

# Applying Word2Vec on feature

In [14]:
from gensim.models import Word2Vec
import numpy as np

# Train Word2Vec model on the tokenized emails
w2v_model = Word2Vec(sentences=data['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Function to get average vector for an email
def get_email_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Apply the vectorization to each email
data['email_vector'] = data['tokens'].apply(lambda x: get_email_vector(x, w2v_model))

# Show the first email's vector
data['email_vector'].iloc[0]

array([-0.53958267,  0.15967034,  0.20131621,  0.04626382,  0.39282158,
       -0.7342825 ,  0.10176148,  0.38037777, -0.47698653,  0.24191089,
        0.11617458, -0.2516565 ,  0.18989307,  1.0051484 , -0.17492169,
       -0.81920904,  0.13157633, -0.4800225 , -0.09694902, -0.88729644,
        0.18532608,  0.5545453 ,  0.2858408 , -0.6760329 , -0.34603927,
       -0.21895857,  0.04411529, -0.33266634, -0.89029926, -0.6720351 ,
       -0.30490717,  0.5023381 , -0.27007383,  0.2649097 , -0.11068711,
        0.6127469 , -0.02654595, -0.27194387, -0.43797716, -0.72538185,
       -0.2784846 , -0.37678176,  0.15818031,  0.4216244 ,  0.98564774,
        0.57315046, -0.49364743,  0.36548972,  0.704587  ,  0.6819436 ,
       -0.57987344, -0.55891067,  0.20516986,  0.43641305, -0.9079877 ,
        0.2912165 ,  0.18853815, -0.29809293,  0.09953497, -0.42889696,
        1.2687397 ,  0.1418594 , -0.21145517, -0.7539105 , -0.61981833,
        0.12928452, -0.67066675, -0.20522009, -1.0639485 ,  0.26

# Apply the Tfidf and Bag of Words for the input text feature embeddings

In [15]:
# tfidf works on the string not list so,
data['cleaned_text'] = data['tokens'].apply(lambda tokens: ' '.join(tokens))

In [16]:
data['cleaned_text']

0        disc uniformitarian sex lang dick hudson obser...
1        side galicismo galicismo spanish term name imp...
2        equistar deal ticket still avail assist robert...
3        hello hot lil horni toy one dream open mind pe...
4        softwar incred low price lower draperi sevente...
                               ...                        
18645    date lone housewif alway want date lone housew...
18646    request submit access request anita dupont enr...
18647    import prc mtg hi dorn john discov recent stil...
18648    press clip letter californian util pleas find ...
18649                                                empti
Name: cleaned_text, Length: 18650, dtype: object

In [17]:
# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
BOW=CountVectorizer()
Final_vec=BOW.fit_transform(data['cleaned_text'])
# Convert sparse matrix row by row to list and store


In [18]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
X_BOW = svd.fit_transform(Final_vec)  # Now shape = (18650, 300

In [19]:
X_BOW.shape

(18650, 300)

In [20]:
data['Bow_embed'] = list(X_BOW)

In [21]:
data[['Bow_embed']].shape

(18650, 1)

# Applying TFIDF

In [22]:
# TFidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
Final_embed=tfidf.fit_transform(data['cleaned_text'])


In [23]:
Final_embed.shape

(18650, 135575)

In [24]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
X_reduced = svd.fit_transform(Final_embed)  # Now shape = (18650, 300

In [25]:
X_reduced.shape

(18650, 300)

In [26]:
# Store TF-IDF vectors
data['tfidf_vector'] = list(X_reduced)

In [27]:
data['tfidf_vector'].shape

(18650,)

# Target Feature

In [28]:
data['Email Type'].value_counts(normalize=True)

Email Type
Safe Email        0.607078
Phishing Email    0.392922
Name: proportion, dtype: float64

In [29]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
email_type_encoded = encoder.fit_transform(data[['Email Type']])

# Convert to DataFrame for easy viewing
email_type_encoded_df = pd.DataFrame(email_type_encoded, columns=encoder.get_feature_names_out(['Email Type']))

# Concatenate with the original data if needed
data = pd.concat([data.reset_index(drop=True), email_type_encoded_df.reset_index(drop=True)], axis=1)

# Show the result
data.head()

,Unnamed: 0,Email Text,Email Type,tokens,email_vector,cleaned_text,Bow_embed,tfidf_vector,Email Type_Phishing Email,Email Type_Safe Email
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email,"[disc, uniformitarian, sex, lang, dick, hudson...","[-0.53958267, 0.15967034, 0.20131621, 0.046263...",disc uniformitarian sex lang dick hudson obser...,"[1.799725152866821, 0.38794364043816976, -0.24...","[0.0005580416963623964, 0.09553076887316486, -...",0.0,1.0
1,1,the other side of * galicismos * * galicismo *...,Safe Email,"[side, galicismo, galicismo, spanish, term, na...","[-0.556427, 0.57890993, 0.2551161, 0.052174464...",side galicismo galicismo spanish term name imp...,"[0.8730081035611174, 0.07544923551420193, 0.00...","[0.00025691307289505983, 0.051836051283405515,...",0.0,1.0
2,2,re : equistar deal tickets are you still avail...,Safe Email,"[equistar, deal, ticket, still, avail, assist,...","[-0.90086126, 1.0569011, -0.48077336, 1.698711...",equistar deal ticket still avail assist robert...,"[1.874636039877039, 1.462161549685051, 8.02190...","[0.0005051338138427337, 0.1467612801331745, 0....",0.0,1.0
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email,"[hello, hot, lil, horni, toy, one, dream, open...","[-0.45548648, 0.15703426, -0.5517961, 0.138889...",hello hot lil horni toy one dream open mind pe...,"[0.9673445770644032, 0.07559244934972142, -0.0...","[0.0005180361073890305, 0.10430117243200257, -...",1.0,0.0
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email,"[softwar, incred, low, price, lower, draperi, ...","[-0.5969315, 0.33947402, 0.06828917, 0.2932554...",softwar incred low price lower draperi sevente...,"[0.8891661466033508, 0.23350636868705374, -0.1...","[0.00048441727390519514, 0.09442886609457772, ...",1.0,0.0


In [31]:
data.head()

,Unnamed: 0,Email Text,Email Type,tokens,email_vector,cleaned_text,Bow_embed,tfidf_vector,Email Type_Phishing Email,Email Type_Safe Email
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email,"[disc, uniformitarian, sex, lang, dick, hudson...","[-0.53958267, 0.15967034, 0.20131621, 0.046263...",disc uniformitarian sex lang dick hudson obser...,"[1.799725152866821, 0.38794364043816976, -0.24...","[0.0005580416963623964, 0.09553076887316486, -...",0.0,1.0
1,1,the other side of * galicismos * * galicismo *...,Safe Email,"[side, galicismo, galicismo, spanish, term, na...","[-0.556427, 0.57890993, 0.2551161, 0.052174464...",side galicismo galicismo spanish term name imp...,"[0.8730081035611174, 0.07544923551420193, 0.00...","[0.00025691307289505983, 0.051836051283405515,...",0.0,1.0
2,2,re : equistar deal tickets are you still avail...,Safe Email,"[equistar, deal, ticket, still, avail, assist,...","[-0.90086126, 1.0569011, -0.48077336, 1.698711...",equistar deal ticket still avail assist robert...,"[1.874636039877039, 1.462161549685051, 8.02190...","[0.0005051338138427337, 0.1467612801331745, 0....",0.0,1.0
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email,"[hello, hot, lil, horni, toy, one, dream, open...","[-0.45548648, 0.15703426, -0.5517961, 0.138889...",hello hot lil horni toy one dream open mind pe...,"[0.9673445770644032, 0.07559244934972142, -0.0...","[0.0005180361073890305, 0.10430117243200257, -...",1.0,0.0
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email,"[softwar, incred, low, price, lower, draperi, ...","[-0.5969315, 0.33947402, 0.06828917, 0.2932554...",softwar incred low price lower draperi sevente...,"[0.8891661466033508, 0.23350636868705374, -0.1...","[0.00048441727390519514, 0.09442886609457772, ...",1.0,0.0


In [32]:
data.rename(columns={'email_vector':'Vector Embedding of email','Email Type_Phishing Email':'Email Type-safe-0_phishing-1'},inplace=True)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18650 entries, 0 to 18649
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    18650 non-null  int64  
 1   Email Text                    18650 non-null  object 
 2   Email Type                    18650 non-null  object 
 3   tokens                        18650 non-null  object 
 4   Vector Embedding of email     18650 non-null  object 
 5   cleaned_text                  18650 non-null  object 
 6   Bow_embed                     18650 non-null  object 
 7   tfidf_vector                  18650 non-null  object 
 8   Email Type-safe-0_phishing-1  18650 non-null  float64
 9   Email Type_Safe Email         18650 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 1.4+ MB


# Applying ML models on the data

# Hyperparameter Tuning and Mlflow integration for experiment tracking

In [34]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import mlflow
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
import mlflow.sklearn

In [35]:
models={
    'catboost': {
        'model': CatBoostClassifier(
            iterations=200,
            learning_rate=0.05,
            depth=5,
            verbose=False,
            random_state=42
        ),
        'params': {
            'iterations': [100, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'depth': [3, 5, 7]
        }
    },
    'lightgbm': {
        'model': LGBMClassifier(
            n_estimators=200,
            learning_rate=0.05,
            max_depth=5,
            random_state=42
        ),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7],
            'num_leaves': [15, 31, 63]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [3, 5],
            'learning_rate': [0.1]
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [4, 8]
        }
    },
    'SVC': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1],
            'kernel': ['linear']
        }
    },
    'Decision_Tree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'max_depth':[5,8],
        }
    },
   'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.01, 0.1, 1],
            'penalty': ['l2'],
            'solver': ['lbfgs']
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    }
}

# Word2vec - trained on the input data

In [36]:
Feature = data['Vector Embedding of email']
Target=data['Email Type-safe-0_phishing-1']
X_train,X_test,y_train,y_test=train_test_split(Feature,Target,train_size=0.8,shuffle=True)
X_train = np.vstack(X_train)
X_test= np.vstack(X_test)

In [ ]:

mlflow.set_experiment('Hyper-Parametertuning of Classifier_with_Metrices_Word_2_vec_')
mlflow.set_registry_uri('http://127.0.0.1:5000/')

for model_name,clf_model in models.items():
    with mlflow.start_run(run_name=model_name+' Hyperparameter tuning'):
        classifier=GridSearchCV(clf_model['model'],clf_model['params'],cv=5,scoring='accuracy')
        classifier.fit(X_train,y_train)
        y_pred=classifier.predict(X_test)
        report=classification_report(y_test,y_pred,output_dict=True)
        mlflow.log_param("model_name",model_name)

        #log parameters
        for param_name, param_value in classifier.best_params_.items():
            mlflow.log_param(param_name,param_value)
        
        mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_class_0': report['0.0']['recall'],
        'recall_class_1': report['1.0']['recall'],
        'f1_score_macro': report['macro avg']['f1-score']
         })
        # Log best model
        mlflow.sklearn.log_model(classifier.best_estimator_, model_name + "_model")
        

        print(f"{model_name} logged in MLflow with accuracy: {classifier.best_score_:.4f}")



# TFIDF

In [38]:
Feature = data['tfidf_vector']
Target=data['Email Type-safe-0_phishing-1']
X_train,X_test,y_train,y_test=train_test_split(Feature,Target,train_size=0.8,shuffle=True)
X_train = np.vstack(X_train)
X_test= np.vstack(X_test)

In [39]:

mlflow.set_experiment('Hyper-Parametertuning of Classifier_with_Metrices_TFIDF_')
mlflow.set_registry_uri('http://127.0.0.1:5000/')

for model_name,clf_model in models.items():
    with mlflow.start_run(run_name=model_name+' Hyperparameter tuning'):
        classifier=GridSearchCV(clf_model['model'],clf_model['params'],cv=5,scoring='accuracy')
        classifier.fit(X_train,y_train)
        y_pred=classifier.predict(X_test)
        report=classification_report(y_test,y_pred,output_dict=True)
        mlflow.log_param("model_name",model_name)

        #log parameters
        for param_name, param_value in classifier.best_params_.items():
            mlflow.log_param(param_name,param_value)
        
        mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_class_0': report['0.0']['recall'],
        'recall_class_1': report['1.0']['recall'],
        'f1_score_macro': report['macro avg']['f1-score']
         })
        # Log best model
        mlflow.sklearn.log_model(classifier.best_estimator_, model_name + "_model")
        

        print(f"{model_name} logged in MLflow with accuracy: {classifier.best_score_:.4f}")



2025/08/01 10:50:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/01 10:50:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


catboost logged in MLflow with accuracy: 0.9617
[LightGBM] [Info] Number of positive: 4688, number of negative: 7248
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043157 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 11936, number of used features: 300
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.392761 -> initscore=-0.435720
[LightGBM] [Info] Start training from score -0.435720
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

d:\Model_Deployement\End_to_End_Phishing_mail_detection_ML_project\nlp\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 4688, number of negative: 7248
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047775 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 11936, number of used features: 300
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.392761 -> initscore=-0.435720
[LightGBM] [Info] Start training from score -0.435720
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

d:\Model_Deployement\End_to_End_Phishing_mail_detection_ML_project\nlp\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

Decision_Tree logged in MLflow with accuracy: 0.9244


2025/08/01 11:14:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/01 11:14:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LogisticRegression logged in MLflow with accuracy: 0.9545


2025/08/01 11:14:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/01 11:14:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


KNN logged in MLflow with accuracy: 0.8932


# BOW

In [40]:
Feature = data['Bow_embed']
Target=data['Email Type-safe-0_phishing-1']
X_train,X_test,y_train,y_test=train_test_split(Feature,Target,train_size=0.8,shuffle=True)
X_train = np.vstack(X_train)
X_test= np.vstack(X_test)

In [ ]:

mlflow.set_experiment('Hyper-Parametertuning of Classifier_with_Metrices_BOW_')
mlflow.set_registry_uri('http://127.0.0.1:5000/')

for model_name,clf_model in models.items():
    with mlflow.start_run(run_name=model_name+' Hyperparameter tuning'):
        classifier=GridSearchCV(clf_model['model'],clf_model['params'],cv=5,scoring='accuracy')
        classifier.fit(X_train,y_train)
        y_pred=classifier.predict(X_test)
        report=classification_report(y_test,y_pred,output_dict=True)
        mlflow.log_param("model_name",model_name)

        #log parameters
        for param_name, param_value in classifier.best_params_.items():
            mlflow.log_param(param_name,param_value)
        
        mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_class_0': report['0.0']['recall'],
        'recall_class_1': report['1.0']['recall'],
        'f1_score_macro': report['macro avg']['f1-score']
         })
        # Log best model
        mlflow.sklearn.log_model(classifier.best_estimator_, model_name + "_model")
        

        print(f"{model_name} logged in MLflow with accuracy: {classifier.best_score_:.4f}")

